In [19]:
#Cell with all the imports
import tensorflow as tf
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import lyricsgenius
import csv
from transformers import TextDataset,DataCollatorForLanguageModeling
import pandas as pd
from sklearn.model_selection import train_test_split

In [49]:
#Scraping for song_lyrics
def get_lyrics(artist_name, max_songs = float('inf')):
    genius = lyricsgenius.Genius('l8-nnpE9RfPvXrIpXq1zhiO1SOdcwCD5m3sOUw9rZlrtWRY1f-9K4wD5tdWK2xN6')
    genius.timeout = 15
    artist = genius.search_artist(artist_name, max_songs=max_songs, sort='popularity')
    print(artist)
    songs = []
    for song in artist.songs:
        songs.append(song.lyrics)
    return songs

def get_lyrics_and_save(artist_name, max_songs = float('inf')):
    songs = get_lyrics(artist_name, max_songs)
    with open(f'{artist_name}.txt', 'w') as file:
        for string in songs:
            file.write(string + '\n')
    return songs

vale = get_lyrics_and_save("Drake", 3)
vale

Searching for songs by Drake...

Song 1: "God’s Plan"
Song 2: "In My Feelings"
Song 3: "Hotline Bling"

Reached user-specified song limit (3).
Done. Found 3 songs.
Drake, 3 songs


['633 ContributorsTranslationsРусскийTürkçeEspañolPolskiItalianoDeutschFrançaisAzərbaycanca / آذربايجانGod’s Plan Lyrics[Intro]\nAnd they wishin\' and wishin\'\nAnd wishin\' and wishin\', they wishin\' on me\nYeah\n\n[Verse 1]\nI been movin\' calm, don\'t start no trouble with me\nTryna keep it peaceful is a struggle for me\nDon\'t pull up at 6 AM to cuddle with me\nYou know how I like it when you lovin\' on me\nI don\'t wanna die for them to miss me\nYes, I see the things that they wishin\' on me\nHope I got some brothers that outlive me\nThey gon\' tell the story, shit was different with me\n\n[Chorus]\nGod\'s plan, God\'s plan\nI hold back, sometimes I won\'t, yeah\nI feel good, sometimes I don\'t (Ayy, don\'t)\nI finessed down Weston Road (Ayy, \'nessed)\nMight go down a G.O.D. (Yeah, wait)\nI go hard on Southside G (Yeah, wait)\nI make sure that north-side eat\nAnd still\n\n[Post-Chorus]\nBad things\nIt\'s a lot of bad things that they wishin\' and wishin\'\nAnd wishin\' and wishi

In [50]:
#Load GPT2 Model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, pad_token_id = tokenizer.eos_token_id)

In [51]:
#Finetune here before usage of model:
#Testing with CSV with ED_SHEERAN SONGS: 
def read_dataset(artist_name, tokenizer):
    train_dataset = TextDataset(file_path="data/Drake.txt",
                                block_size=128,
                                tokenizer=tokenizer)
    

    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False,)
    return train_dataset, data_collator

train_dataset, data_collator = read_dataset("ed_sheeran", tokenizer)

In [52]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=6, training_loss=3.814711888631185, metrics={'train_runtime': 20.5172, 'train_samples_per_second': 2.632, 'train_steps_per_second': 0.292, 'total_flos': 3527442432000.0, 'train_loss': 3.814711888631185, 'epoch': 3.0})

In [43]:
# Save the trained model
output_dir = "./models/drake"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./models/joe_cole/tokenizer_config.json',
 './models/joe_cole/special_tokens_map.json',
 './models/joe_cole/vocab.json',
 './models/joe_cole/merges.txt',
 './models/joe_cole/added_tokens.json')

In [53]:
#Encoding the input
start_of_song = "The world is dizzy"
input_ids = tokenizer.encode(start_of_song, return_tensors="pt")

In [54]:
#Decoding the input again to see if it works
tokenizer.decode(input_ids[0])

'The world is dizzy'

In [55]:
output = model.generate(input_ids, max_length=500, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)

In [56]:
output

tensor([[  464,   995,   318, 29527,    88,    13,   198,   198,  1026,   338,
           640,   329,   345,   284,   651,   503,   612,   290,   466,  1223,
           546,   340,    13, 50256]])

In [57]:
tokenizer.decode(output[0])

"The world is dizzy.\n\nIt's time for you to get out there and do something about it.<|endoftext|>"